<a href="https://colab.research.google.com/github/LordHarsh/Face_Detection/blob/main/Face_Recognition_v3a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Cloning repo to get other data

In [ ]:
!git clone https://github.com/LordHarsh/Face_Detection
%cd /content/Face_Detection/

# Face Recognition

Here, we will build a face recognition system. Many of the ideas presented here are from [FaceNet](https://arxiv.org/pdf/1503.03832.pdf) and [DeepFace](https://research.fb.com/wp-content/uploads/2016/11/deepface-closing-the-gap-to-human-level-performance-in-face-verification.pdf).

Face recognition problems commonly fall into two categories: 

- **Face Verification** - "is this the claimed person?". For example, at some airports, you can pass through customs by letting a system scan your passport and then verifying that you (the person carrying the passport) are the correct person. A mobile phone that unlocks using your face is also using face verification. This is a 1:1 matching problem. 
- **Face Recognition** - "who is this person?". For example, the  employees entering the office without needing to otherwise identify themselves. This is a 1:K matching problem. 

FaceNet learns a neural network that encodes a face image into a vector of 128 numbers. By comparing two such vectors, you can then determine if two pictures are of the same person.
    
**Here, we will:**
- Implement the triplet loss function
- Use a pretrained model to map face images into 128-dimensional encodings
- Use OpenCV to detect faces
- Use these encodings to perform face verification and face recognition

#### Channels-first notation

* Here, we will be using a pre-trained model which represents ConvNet activations using a **"channels first"** convention, as opposed to the "channels last" convention. 
* In other words, a batch of images will be of shape $(m, n_C, n_H, n_W)$ instead of $(m, n_H, n_W, n_C)$. 
* Both of these conventions have a reasonable amount of traction among open-source implementations; there isn't a uniform standard yet within the deep learning community. 

#### Load packages
Let's load the required packages. 

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from keras import backend as K
from google.colab import files
K.set_image_data_format('channels_first')
import cv2
import os
import numpy as np
import shutil
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
from fr_utils import *
from inception_blocks_v2 import *
import sys
import PIL
from PIL import Image
%matplotlib inline
%load_ext autoreload
%autoreload 2

np.set_printoptions(threshold=sys.maxsize)

## 0 - Naive Face Verification

In Face Verification, we're given two images and you have to determine if they are of the same person. The simplest way to do this is to compare the two images pixel-by-pixel. If the distance between the raw images are less than a chosen threshold, it may be the same person! 


* Of course, this algorithm performs really poorly, since the pixel values change dramatically due to variations in lighting, orientation of the person's face, even minor changes in head position, and so on. 
* You'll see that rather than using the raw image, you can learn an encoding, $f(img)$.  
* By using an encoding for each image, an element-wise comparison produces a more accurate judgement as to whether two pictures are of the same person.

## 1 - Encoding face images into a 128-dimensional vector 

### 1.1 - Using a ConvNet  to compute encodings

The FaceNet model takes a lot of data and a long time to train. So following common practice in applied deep learning, let's  load weights that someone else has already trained. The network architecture follows the Inception model from [Szegedy *et al.*](https://arxiv.org/abs/1409.4842). We have provided an inception network implementation. You can look in the file `inception_blocks_v2.py` to see how it is implemented (do so by going to "File->Open..." at the top of the Jupyter notebook.  This opens the file directory that contains the '.py' file). 

The key things you need to know are:

- This network uses 96x96 dimensional RGB images as its input. Specifically, inputs a face image (or batch of $m$ face images) as a tensor of shape $(m, n_C, n_H, n_W) = (m, 3, 96, 96)$ 
- It outputs a matrix of shape $(m, 128)$ that encodes each input face image into a 128-dimensional vector

Run the cell below to create the model for face images.

In [ ]:
FRmodel = faceRecoModel(input_shape=(3, 96, 96))

In [ ]:
print("Total Params:", FRmodel.count_params())

**Expected Output**
<table>
<center>
Total Params: 3743280
</center>
</table>


By using a 128-neuron fully connected layer as its last layer, the model ensures that the output is an encoding vector of size 128. You then use the encodings to compare two face images.


So, an encoding is a good one if: 
- The encodings of two images of the same person are quite similar to each other. 
- The encodings of two images of different persons are very different.

The triplet loss function formalizes this, and tries to "push" the encodings of two images of the same person (Anchor and Positive) closer together, while "pulling" the encodings of two images of different persons (Anchor, Negative) further apart. 




### 1.2 - The Triplet Loss

For an image $x$, we denote its encoding $f(x)$, where $f$ is the function computed by the neural network.


<!--
We will also add a normalization step at the end of our model so that $\mid \mid f(x) \mid \mid_2 = 1$ (means the vector of encoding should be of norm 1).
!-->

Training will use triplets of images $(A, P, N)$:  

- A is an "Anchor" image--a picture of a person. 
- P is a "Positive" image--a picture of the same person as the Anchor image.
- N is a "Negative" image--a picture of a different person than the Anchor image.

These triplets are picked from our training dataset. We will write $(A^{(i)}, P^{(i)}, N^{(i)})$ to denote the $i$-th training example. 

You'd like to make sure that an image $A^{(i)}$ of an individual is closer to the Positive $P^{(i)}$ than to the Negative image $N^{(i)}$) by at least a margin $\alpha$:

$$\mid \mid f(A^{(i)}) - f(P^{(i)}) \mid \mid_2^2 + \alpha < \mid \mid f(A^{(i)}) - f(N^{(i)}) \mid \mid_2^2$$

You would thus like to minimize the following "triplet cost":

$$\mathcal{J} = \sum^{m}_{i=1} \large[ \small \underbrace{\mid \mid f(A^{(i)}) - f(P^{(i)}) \mid \mid_2^2}_\text{(1)} - \underbrace{\mid \mid f(A^{(i)}) - f(N^{(i)}) \mid \mid_2^2}_\text{(2)} + \alpha \large ] \small_+ \tag{3}$$

Here, we are using the notation "$[z]_+$" to denote $max(z,0)$.  

Notes:
- The term (1) is the squared distance between the anchor "A" and the positive "P" for a given triplet; you want this to be small. 
- The term (2) is the squared distance between the anchor "A" and the negative "N" for a given triplet, you want this to be relatively large. It has a minus sign preceding it because minimizing the negative of the term is the same as maximizing that term.
- $\alpha$ is called the margin. It is a hyperparameter that you pick manually. We will use $\alpha = 0.2$. 

Most implementations also rescale the encoding vectors to haven L2 norm equal to one (i.e., $\mid \mid f(img)\mid \mid_2$=1); you won't have to worry about that in this assignment.

Implement the triplet loss as defined by formula (3). Here are the 4 steps:
1. Compute the distance between the encodings of "anchor" and "positive": $\mid \mid f(A^{(i)}) - f(P^{(i)}) \mid \mid_2^2$
2. Compute the distance between the encodings of "anchor" and "negative": $\mid \mid f(A^{(i)}) - f(N^{(i)}) \mid \mid_2^2$
3. Compute the formula per training example: $ \mid \mid f(A^{(i)}) - f(P^{(i)}) \mid \mid_2^2 - \mid \mid f(A^{(i)}) - f(N^{(i)}) \mid \mid_2^2 + \alpha$
3. Compute the full formula by taking the max with zero and summing over the training examples:
$$\mathcal{J} = \sum^{m}_{i=1} \large[ \small \mid \mid f(A^{(i)}) - f(P^{(i)}) \mid \mid_2^2 - \mid \mid f(A^{(i)}) - f(N^{(i)}) \mid \mid_2^2+ \alpha \large ] \small_+ \tag{3}$$

#### Hints
* Useful functions: `tf.reduce_sum()`, `tf.square()`, `tf.subtract()`, `tf.add()`, `tf.maximum()`.
* For steps 1 and 2, you will sum over the entries of $\mid \mid f(A^{(i)}) - f(P^{(i)}) \mid \mid_2^2$ and $\mid \mid f(A^{(i)}) - f(N^{(i)}) \mid \mid_2^2$.  
* For step 4 you will sum over the training examples.
* Recall that the square of the L2 norm is the sum of the squared differences: $||x - y||_{2}^{2} = \sum_{i=1}^{N}(x_{i} - y_{i})^{2}$
* Note that the `anchor`, `positive` and `negative` encodings are of shape `(m,128)`, where m is the number of training examples and 128 is the number of elements used to encode a single example.
* For steps 1 and 2, you will maintain the number of `m` training examples and sum along the 128 values of each encoding. 
[tf.reduce_sum](https://www.tensorflow.org/api_docs/python/tf/math/reduce_sum) has an `axis` parameter.  This chooses along which axis the sums are applied.  
* Note that one way to choose the last axis in a tensor is to use negative indexing (`axis=-1`).
* In step 4, when summing over training examples, the result will be a single scalar value.
* For `tf.reduce_sum` to sum across all axes, keep the default value `axis=None`.

In [ ]:
def triplet_loss(y_true, y_pred, alpha = 0.2):
    """
    Implementation of the triplet loss as defined by formula (3)
    
    Arguments:
    y_true -- true labels, required when you define a loss in Keras, you don't need it in this function.
    y_pred -- python list containing three objects:
            anchor -- the encodings for the anchor images, of shape (None, 128)
            positive -- the encodings for the positive images, of shape (None, 128)
            negative -- the encodings for the negative images, of shape (None, 128)
    
    Returns:
    loss -- real number, value of the loss
    """
    
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    
    ### START CODE HERE ### (≈ 4 lines)
    # Step 1: Compute the (encoding) distance between the anchor and the positive
    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, positive)), axis=-1)
    # Step 2: Compute the (encoding) distance between the anchor and the negative
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, negative)), axis=-1)
    # Step 3: subtract the two previous distances and add alpha.
    basic_loss = tf.subtract(pos_dist, neg_dist) + alpha
    # Step 4: Take the maximum of basic_loss and 0.0. Sum over the training examples.
    loss = tf.reduce_sum(tf.maximum(basic_loss,0))
    ### END CODE HERE ###
    
    return loss

In [ ]:
with tf.compat.v1.Session() as test:
    y_true = (None, None, None)
    y_pred = (tf.random.normal([3, 128], mean=6, stddev=0.1, seed = 1),
              tf.random.normal([3, 128], mean=1, stddev=1, seed = 1),
              tf.random.normal([3, 128], mean=3, stddev=4, seed = 1))
    loss = triplet_loss(y_true, y_pred)
    
    print("loss = " + str(loss.eval()))

## 2 - Loading the pre-trained model

FaceNet is trained by minimizing the triplet loss. But since training requires a lot of data and a lot of computation, we won't train it from scratch here. Instead, we load a previously trained model. Load a model using the following cell; this might take a couple of minutes to run. 

In [ ]:
FRmodel.compile(optimizer = 'adam', loss = triplet_loss, metrics = ['accuracy'])
load_weights_from_FaceNet(FRmodel)

## 3 - Applying the model

We are building a system for an office building where the building manager  would like to offer facial recognition to allow the employees to enter the building.

We'd like to build a **Face verification** system that gives access to the list of people who live or work there. To get admitted, each person has to swipe an ID card (identification card) to identify themselves at the entrance. The face recognition system then checks that they are who they claim to be.

### 3.1 - Face Verification

Let's build a database containing one encoding vector for each person who is allowed to enter the office. To generate the encoding we use `img_to_encoding(image_path, model)`, which runs the forward propagation of the model on the specified image. 


In [ ]:
def verify(image_path, identity, database, model):
    """
    Function that verifies if the person on the "image_path" image is "identity".
    
    Arguments:
    image_path -- path to an image
    identity -- string, name of the person you'd like to verify the identity. Has to be an employee who works in the office.
    database -- python dictionary mapping names of allowed people's names (strings) to their encodings (vectors).
    model -- your Inception model instance in Keras
    
    Returns:
    dist -- distance between the image_path and the image of "identity" in the database.
    door_open -- True, if the door should open. False otherwise.
    """
    
    
    # Step 1: Compute the encoding for the image. Use img_to_encoding() see example above. (≈ 1 line)
    encoding = img_to_encoding(image_path, model)
    
    # Step 2: Compute distance with identity's image (≈ 1 line)
    dist = np.linalg.norm(encoding-database[identity])
    
    # Step 3: Open the door if dist < 0.7, else don't open (≈ 3 lines)
    if dist<0.7:
        print("It's " + str(identity) + ", welcome in!")
        door_open = True
    else:
        print("It's not " + str(identity) + ", please go away")
        door_open = False
        
        
    return dist, door_open

### 3.2 - Face Recognition

Our face verification system is mostly working well. But when a person got his ID card stolen and cames back to the office the next day, he couldn't get in! 

To solve this, we'd like to change your face verification system to a face recognition system. This way, no one has to carry an ID card anymore. An authorized person can just walk up to the building, and the door will unlock for them! 

We'll implement a face recognition system that takes as input an image, and figures out if it is one of the authorized persons (and if so, who). Unlike the previous face verification system, we will no longer get a person's name as one of the inputs. 
Implement `who_is_it()`. We will have to go through the following steps:
1. Compute the target encoding of the image from image_path
2. Find the encoding from the database that has smallest distance with the target encoding. 
    - Initialize the `min_dist` variable to a large enough number (100). It will help you keep track of what is the closest encoding to the input's encoding.
    - Loop over the database dictionary's names and encodings. To loop use `for (name, db_enc) in database.items()`.
        - Compute the L2 distance between the target "encoding" and the current "encoding" from the database.
        - If this distance is less than the min_dist, then set `min_dist` to `dist`, and `identity` to `name`.

In [ ]:
def who_is_it(image_path, database, model):
    """
    Implements face recognition for the office by finding who is the person on the image_path image.
    
    Arguments:
    image_path -- path to an image
    database -- database containing image encodings along with the name of the person on the image
    model -- your Inception model instance in Keras
    
    Returns:
    min_dist -- the minimum distance between image_path encoding and the encodings from the database
    identity -- string, the name prediction for the person on image_path
    """
    

    
    ## Step 1: Compute the target "encoding" for the image. Use img_to_encoding() see example above. ## (≈ 1 line)
    encoding = img_to_encoding(image_path, model)
    
    ## Step 2: Find the closest encoding ##
    
    # Initialize "min_dist" to a large value, say 100 (≈1 line)
    min_dist = 100
    
    # Loop over the database dictionary's names and encodings.
    for (name, db_enc) in database.items():
        
        # Compute L2 distance between the target "encoding" and the current db_enc from the database. (≈ 1 line)
        dist = np.linalg.norm(encoding-db_enc)

        # If this distance is less than the min_dist, then set min_dist to dist, and identity to name. (≈ 3 lines)
        if dist < min_dist:
            min_dist = dist
            identity = name


    
    if min_dist > 0.7:
        print("Not in the database.")
    else:
        print ("it's " + str(identity) + ", the distance is " + str(min_dist))
        
    return min_dist, identity

## 4 - Saving image after face detection and resizing

### 4.1 - Face Detection

Here. we use Opencv2 for face detection. ```haarcascade_frontalface_default.xml``` is the file with tells cv2 which object to detect.

In [ ]:
from urllib.request import urlopen
def url_to_image(url, readFlag=cv2.IMREAD_COLOR):
    # download the image, convert it to a NumPy array, and then read
    # it into OpenCV format
    resp = urlopen(url)
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    image = cv2.imdecode(image, readFlag)

    # return the image
    return image
def cassscade(imagePath, cascPath):
  if ('http' in imagePath) or ('www' in imagePath):
    image = url_to_image(imagePath)
  else:
    image = cv2.imread(imagePath)
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  # Detect faces in the image
  faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + cascPath)
  faces = faceCascade.detectMultiScale(
      gray,
      scaleFactor=1.3,
      minNeighbors=5,
      minSize=(150,150),
      flags = cv2.CASCADE_SCALE_IMAGE
  )
  print ("Found {0} faces!".format(len(faces)))
  return faces, image

In [ ]:
def get_crop_face(img_path):
  faces, image = cassscade(img_path,'haarcascade_frontalface_default.xml')
  [[y, x, width, height]] = faces
  x1 = max((int)(x-(width*0.1)),0)
  x2 = min((int)(x+width+(width*0.1)),image.shape[0])
  y1 = max((int)(y-(height*0.1)),0)
  y2 = min((int)(y+height+(height*0.1)),image.shape[1])
  crop_img = image[x1:x2,y1:y2].copy()
  im_shape = np.array(crop_img.shape)
  ratio = float(224) / min(im_shape[0],im_shape[1])     # Calculating the ratio to be changed
  crop_img = cv2.cvtColor(crop_img, cv2.COLOR_RGB2BGR)  # Manging color shifts
  crop_img_pil = Image.fromarray(crop_img)
  new_shape=((int)(np.ceil(im_shape[0] * ratio)), (int)(np.ceil(im_shape[1] * ratio)))
  resize_img_pil = crop_img_pil.resize(size=(96,96), resample=PIL.Image.BILINEAR)
  return(resize_img_pil)
#def get_crop_face_from_url(url):
# if()

### 4.2 - Getting encoding from Database

Run the following code to build the database (represented as a python dictionary). This database maps each person's name to a 128-dimensional encoding of their face.

In [ ]:
def get_encodings(folder):
    database = {}
    for filen in os.listdir(folder):
      filename=os.path.splitext(filen)[0]
      filepath = os.path.join(folder,filen)
      if(os.path.splitext(filen)[1]=='.jpg'):
        database[filename]=img_to_encoding(filepath, FRmodel)
    return database

In [ ]:
database = get_encodings('/content/Face_Detection/database_dir')
def add_to_database(name, filepath, database):
  database[name]=img_to_encoding(filepath, FRmodel)
  return database

## 5 - Start the main work

<font color='greenl'>
To enter the your image in database fill the form given below by:

* Selecting from where to enter picture
* Name of the person
</font>
<font color='greenl'>
Then press ```ctrl+Enter``` to run the form cell.

The image will be saved in the database.
Remember this temporary, this will be deleted once you reload the page.
**Note-** Make sure picture is high resolution and the face is in the center of the image.
</font>

In [ ]:
#@title Enter to database
Enter_from = "None" #@param ["URL", "From_Local_PC", "None"]
Person_Name = "ENTER NAME HERE (if you want to enter)" #@param {type:"string"}
Person_Name = Person_Name.strip().title()
print("Try to select a high resolution picture with face in the center")
if(Enter_from=='URL'):
  URL = input("Enter URL: ")
  img=get_crop_face(URL)
  display(img.resize(size=(200,200), resample=PIL.Image.BILINEAR))
  img.save('/content/Face_Detection/database_dir/'+Person_Name+".jpg")
  database=add_to_database(Person_Name, filepath='/content/Face_Detection/database_dir/'+Person_Name+'.jpg', database=database)
if(Enter_from=='From_Local_PC'):
  print("Upload only one picture")
  uploaded = files.upload()
  for fn in uploaded.keys():
    name=fn
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))
    shutil.move(fn,"temp/"+fn)
    img=get_crop_face("temp/"+fn)
    display(img.resize(size=(200,200), resample=PIL.Image.BILINEAR))
    img.save('/content/Face_Detection/database_dir/'+Person_Name+".jpg")
  database=add_to_database(Person_Name, filepath='/content/Face_Detection/database_dir/'+Person_Name+'.jpg', database=database)



<font color='greenl'>Now its fun time, lets see if algoritm recognizes you.
Fill the form given below to do that.

**Important:** It may be possible that it give bad results but it's worth a try. This is just for fun so currently, the machine learning model used here is a weakly trained model, In future I will add a better one.
</font>

In [ ]:
print("Currently in database-")
for key in database.keys():
  print(key)

In [ ]:
#@title Verift your image

Enter_from = "From_Local_PC" #@param ["URL", "From_Local_PC", "None"]
Person_Name = "check"
Person_Name = Person_Name.strip().title()
print("Try to select a high resolution picture with face in the center")
if(Enter_from=='URL'):
  URL = input("Enter URL: ")
  img=get_crop_face(URL)
  display(img.resize(size=(200,200), resample=PIL.Image.BILINEAR))
  img.save('/content/Face_Detection/temp/'+Person_Name+".jpg")
  who_is_it('/content/Face_Detection/temp/'+Person_Name+".jpg", database, FRmodel)
  
if(Enter_from=='From_Local_PC'):
  print("Upload only one picture")
  uploaded = files.upload()
  for fn in uploaded.keys():
    name=fn
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))
    shutil.move(fn,"temp/"+fn)
    img = get_crop_face("temp/"+fn)
  display(img.resize(size=(200,200), resample=PIL.Image.BILINEAR))
  img.save('/content/Face_Detection/database_dir/'+Person_Name+".jpg")
  who_is_it('/content/Face_Detection/temp/'+Person_Name+".jpg", database, FRmodel)


##### If the result is wrong
* If the result is wrong then it's not anything wrong our code or your method.
* It's because it machine learning model used here to recognize faces is a weekly trained. This is due to current lack of computing resourses and face data. This will be fixed in the future.